## RFInder: tools to investigate the presence of RFI in a dataset


In [ ]:
import os, string, sys
import numpy as np
import pyaml, json
from astropy.io import ascii

from IPython.display import HTML, display

import rfinder as rfinder

print '\t+------+\n\t  Done \n\t+------+'

### Load and modify parameter file

In [ ]:
print '\t+------+\n\t RFInder \n\t Parameter File \n\t+------+'

rfi_par = rfinder.rfinder('/Users/maccagni/notebooks/rfinder/rfinder_default.yml')

print '\t+------+\n\t Parameters Loaded \n\t+------+'

#print parameter file 
print(json.dumps(rfi.cfg_par, indent=4, sort_keys=True))


In [ ]:
# Set working directory & filenames
rfi_par.cfg_par['general']['workdir'] = '/Users/maccagni/notebooks/rfinder/examples/'

# Enable / disable modules
rfi_par.cfg_par['rfi']['enable'] = True
rfi_par.cfg_par['beam_shape']['enable'] = True
rfi_par.cfg_par['plots']['enable'] = True

rfi_par.set_dirs()
# Options

print '\t+------+\n\t Parameters modified \n\t+------+'


### Investigate presence of RFI in a dataset

In [ ]:
import rfi

rfi = rfi.rfi()

help(rfi.baselines_from_ms)

#find sources
rfi_values = rfi.baselines_from_ms(rfi_par.cfg_par)

#load and print output table
#tot = ascii.read(rfi_par.cfg_par['general']['absdir']+'mir_src_sharpener.csv' )
#display(HTML(tabulate.tabulate(tot, tablefmt='html',headers="keys")))


print '\t+------+\n\t Done \n\t+------+'


### Determine shape of PSF after RFI flagging

In [ ]:
import rfi_beam as rfi_beam

beam = rfi_beam.make_psf(rfi_par.cfg_par)

print '\t+------+\n\t Spectra extracted \n\t+------+'


### Plot RFI

In [ ]:
import glob
import rfi_plots as rfi_pl


#plot spectra
rfi_plots.plot_rfi_im(rfi_par.cfg_par)
rfi_plots.plot_rfi_frequency(rfi_par.cfg_par)



print '\t+------+\n\t Plots done \n\t+------+'
